This model predicts the frequency of making fully joint trips. The household chooses the joint tour purposes and each household can make up to two joint tours

In [15]:
import os
import toml
import pandas as pd
import numpy as np
import psrc_theme
import plotly.express as px
from plotly.subplots import make_subplots

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

# run get_data.ipynb to load validation data
%store -r validation_data

In [13]:
hh_data = validation_data.hh_data_uncloned.copy()
tour_data = validation_data.tours.copy()

# all joint tours
joint_tour_data = tour_data.loc[tour_data['tour_category'] == "joint"]
num_hh_joint_tours = joint_tour_data.groupby(['source','household_id']).size().to_frame(). \
    rename(columns={0: 'num_hh_joint_tours'})

# all households with joint tours
hh_data = pd.merge(hh_data, num_hh_joint_tours, on="household_id")
joint_hh_data = hh_data.loc[hh_data['num_hh_joint_tours']>0]

In [16]:
df_plot = hh_data.groupby(['source','num_hh_joint_tours'])[['hh_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['hh_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="num_hh_joint_tours", y="percentage", color="source",barmode="group", 
             # color_discrete_sequence=config['psrc_color'],
             title="number of household joint tours")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".2%"), xaxis=dict(dtick="1"))
fig.show()

In [17]:
# "joint_tour_frequency" variable not in data

# df_plot = joint_hh_data.groupby(['source','num_hh_joint_tours','joint_tour_frequency'])[['hh_weight']].sum().reset_index()
# df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['hh_weight']. \
#     apply(lambda x: x / float(x.sum()))
# 
# fig = px.bar(df_plot, x="joint_tour_frequency", y="percentage", color="source",barmode="group", template="simple_white",
#              facet_row='num_hh_joint_tours',facet_row_spacing=0.1,
#              title="joint tour frequency (joint tour purpose specified)")
# fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
# fig.update_layout(height=500, width=850, font=dict(size=11),
#                   xaxis1=dict(matches=None,showticklabels=True), xaxis2=dict(matches=None,showticklabels=True),
#                   yaxis1=dict(tickformat=".2%"), yaxis2=dict(tickformat=".2%", matches=None))
# fig.show()

KeyError: 'joint_tour_frequency'